# Fetch Rewards Coding Exercise - Data Analyst

In [83]:
import pandas as pd
from ast import literal_eval
import json
import datetime
from sqlalchemy import create_engine
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

## Read in Data
Read in Receipts, Brands and Users data

In [84]:
receipts = pd.read_json('receipts.json',lines=True)

In [85]:
#receipt
receipts = receipts.fillna({'rewardsReceiptItemList': '{}', 'finishedDate': '{}', 'pointsAwardedDate': '{}', 'purchaseDate': '{}'})

In [86]:
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},{},{'$date': 1609687542000},{},5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},{},{'$date': 1614594148000},{},25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},{},{'$date': 1614613361873},{},NaN,{},NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},{},{'$date': 1614607657664},{},NaN,{},NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},{},{'$date': 1614604049000},{},25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [87]:
#functions for data cleaning
def clean_column_keep_dic_value(df, col, key):  
    all_list=[]
    for i in range(len(df)):
        if df[col].iloc[i] == '{}': 
            all_list.append('')
        else: 
            all_list.append(df[col].iloc[i][key])
    return all_list

def convert_time(df, col):
    time_list = []
    for i in range(len(df)):
        if df[col].iloc[i] == '{}': 
            time_list.append('')
        else:
            time_list.append(datetime.datetime.fromtimestamp(df[col].iloc[i] / 1e3)) 
    return time_list

In [95]:
# time_list = []
# for i in range(len(df)):
#     if df[col].iloc[i] == '{}': 
#         time_list.append('')
#     else:
#         time_list.append(datetime.datetime.fromtimestamp(df[col].iloc[i] / 1e3)) 

In [96]:
# datetime.datetime.fromtimestamp(receipts['finishedDate'].iloc[2] / 1e3)

In [97]:
# receipts['finishedDate'].iloc[2]

In [91]:
#Remove unnecessary words and clean data
receipts['_id'] = clean_column_keep_dic_value(receipts, '_id', '$oid')
receipts['createDate'] = clean_column_keep_dic_value(receipts, 'createDate', '$date')
receipts['dateScanned'] = clean_column_keep_dic_value(receipts, 'dateScanned', '$date')
receipts['modifyDate'] = clean_column_keep_dic_value(receipts, 'modifyDate', '$date')
receipts['finishedDate'] = clean_column_keep_dic_value(receipts, 'finishedDate', '$date')
receipts['pointsAwardedDate'] = clean_column_keep_dic_value(receipts, 'pointsAwardedDate', '$date')
receipts['purchaseDate'] = clean_column_keep_dic_value(receipts, 'purchaseDate', '$date')

In [92]:
receipts.isna().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate                 0
modifyDate                   0
pointsAwardedDate            0
pointsEarned               510
purchaseDate                 0
purchasedItemCount         484
rewardsReceiptItemList       0
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [93]:
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1609687531000,1609687536000,1609687531000,500.0,1609632000000,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1609687483000,1609687488000,1609687483000,150.0,1609601083000,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,,1609687542000,,5.0,1609632000000,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1609687534000,1609687539000,1609687534000,5.0,1609632000000,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1609687511000,1609687511000,1609687506000,5.0,1609601106000,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,,1614594148000,,25.0,1597622400000,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,1614613361873,1614613361873,,1614613361873,,NaN,,NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,1614607657664,1614607657664,,1614607657664,,NaN,,NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,1614604048000,1614604048000,,1614604049000,,25.0,1597622400000,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [94]:
#convert time from UTC to datetime
receipts['createDate'] = convert_time(receipts, 'createDate')
receipts['dateScanned'] = convert_time(receipts, 'dateScanned')
receipts['modifyDate'] = convert_time(receipts, 'modifyDate')
# receipts['finishedDate'] = convert_time(receipts, 'finishedDate')
receipts['pointsAwardedDate'] = convert_time(receipts, 'pointsAwardedDate')
receipts['purchaseDate'] = convert_time(receipts, 'purchaseDate')

#only keep the date
receipts['createDate'] =  receipts['createDate'].dt.date
receipts['dateScanned'] = receipts['dateScanned'].dt.date
receipts['modifyDate'] = receipts['modifyDate'].dt.date
receipts['finishedDate'] = receipts['finishedDate'].dt.date
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].dt.date
receipts['purchaseDate'] = receipts['purchaseDate'].dt.date

TypeError: unsupported operand type(s) for /: 'str' and 'float'

## Brands

In [ ]:
brands = pd.read_json('brands.json',lines=True)

In [ ]:
#Remove unnecessary words and clean data                     
brands['_id'] = clean_column_keep_dic_value(brands, '_id', '$oid')

## Users

In [ ]:
users = pd.read_json('users.json',lines=True)

In [ ]:
users = users.fillna({'createdDate':'{}', 'lastLogin':'{}'})

In [ ]:
#Remove unnecessary words and clean data
users['_id'] = clean_column_keep_dic_value(users, '_id', '$oid')
users['createdDate'] = clean_column_keep_dic_value(users, 'createdDate', '$date')
users['lastLogin'] = clean_column_keep_dic_value(users, 'lastLogin', '$date')

In [ ]:
#convert time from UTC to datetime
users['createdDate'] = convert_time(users, 'createdDate')
users['lastLogin'] = convert_time(users, 'lastLogin')

#only keep the date
users['createdDate'] = users['createdDate'].dt.date
users['lastLogin'] = users['lastLogin'].dt.date

Clean `rewardsReceiptItemList` column. The first step is to explode the `rewardsReceiptItemList` field as it contains lists of receipts(using literal_eval function from ast package). Then I split each dictionary and get the sub-columns (using the json_normalize function in pandas). At the end I join the new created data with the original receipts data frame.

In [ ]:
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

In [ ]:
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

In [ ]:
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

In [ ]:
receipts_final = pd.merge(receipts, receipts_norm, left_index=True, right_index=True, how='outer')
receipts_final = receipts_final.drop(columns=['rewardsReceiptItemList'],axis=1)

In [ ]:
#extract the data in cpg column from brands
set().union(*(d.keys() for d in brands['cpg']))
brands_norm = pd.json_normalize(brands['cpg'])
brands_norm = brands_norm.add_prefix('cpg.')
brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')
brands_final = brands_final.drop(columns=['cpg'],axis=1)

In [ ]:
users

# Join each table to make sure there are not quality problem when joining tables

In [ ]:
receipts_brands = receipts_final.merge(brands[['brandCode']].drop_duplicates(), left_on='rewardsReceiptItemList_brandCode', right_on='brandCode', how='left')

In [ ]:
receipts_brands

In [ ]:
receipts_brands.brandCode.unique()

In [ ]:
receipts_brands[['brandCode']].isna().sum()

In [ ]:
receipts_brands_users = receipts_brands.merge(users.drop_duplicates(), left_on='userId', right_on='_id', how='left')


In [ ]:
receipts_brands_users.isna().sum()

# Data Quality Check

In [ ]:
users.groupby('_id').count()['lastLogin']

## 1. check null percentage

In [ ]:
def null_percentage_calc(df, show_plot=True):  
    null_percentage = round((df.isna().sum()/len(df))*100, 2).to_frame().rename(columns={0:'percentage'}).sort_values(by='percentage', ascending=False)
    if show_plot == True:
        null_percentage.plot(kind='barh')
    return null_percentage


In [ ]:
null_percentage_calc(brands_final)

In [ ]:
null_percentage_calc(users)

In [ ]:
null_percentage_calc(receipts_final[['index', '_id', 'bonusPointsEarned', 'bonusPointsEarnedReason','createDate', 'dateScanned', 'finishedDate', 
                                     'modifyDate','pointsAwardedDate', 'pointsEarned', 'purchaseDate','purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent', 'userId']])

In [ ]:
null_percentage_calc(receipts_final[[col for col in receipts_final if col.startswith('rewardsReceiptItemList')]], False)

## 2.Check data distribution and find potential outliers

In [ ]:
def plot_col_distribution(df, bins):
    for col in df.columns:
        if ((df[col].dtype) == 'float64'):
            df[col].hist(bins=bins)
            plt.title(col + " Distribution")
            plt.show()
    


def boxplot(df):
    ax = sns.boxplot(data=df, orient="h", palette="Set2")

In [ ]:
plot_col_distribution(receipts_final, 10)

In [ ]:
boxplot(receipts_final)

In [ ]:
#check whether purchasedItemCount and totalSpent have positive correlation
sns.regplot(x="purchasedItemCount", y="totalSpent", data=receipts_final)

In [ ]:
receipts_final.describe()

# Write a query that directly answers a predetermined question from a business stakeholder

Before pulling the data from SQL, I drop the rows where `totalSpent` and `purchasedItemCount` in Receipts is null, because when calculating the average spend of total item purchased, filling in null values will make the analysis inaccurate. 

In [ ]:
receipts_final = receipts_final.dropna(subset=['totalSpent', 'purchasedItemCount'], how='all')
receipts_final = receipts_final.dropna(subset=['purchasedItemCount'], how='all')

Connect with SQLite database through SQL Alchemy python (https://docs.sqlalchemy.org/en/14/dialects/sqlite.html) to implement the SQL database for the queries.

In [ ]:
engine = create_engine('sqlite://', echo=False)

In [ ]:
users.drop_duplicates(subset=['_id']).to_sql('users',con=engine)

In [ ]:
receipts_final.columns = receipts_final.columns.str.replace(".", "_")

In [ ]:
receipts_final.to_sql('receipts',con=engine)

In [ ]:
brands_final.drop_duplicates(subset=['brandCode']).to_sql('brands',con=engine)

In [ ]:
receipts.rewardsReceiptStatus.unique()

### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Since I didn't see 'Accepted' as a status in rewardsReceiptStatus, I'm assuming `FINISHED` represents 'Accepted'. I will follow up with stakeholders in the email.

In [ ]:
engine.execute('''SELECT rewardsReceiptStatus, ROUND(AVG(totalSpent), 2) as avg_spent
                  FROM receipts
                  where rewardsReceiptStatus = 'FINISHED' or  rewardsReceiptStatus = 'REJECTED'
                  GROUP BY rewardsReceiptStatus''').fetchall()

### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [ ]:
engine.execute('''SELECT rewardsReceiptStatus, ROUND(SUM(purchasedItemCount), 2) as total_num_purchased
                  FROM receipts
                  where rewardsReceiptStatus = 'FINISHED' or  rewardsReceiptStatus = 'REJECTED'
                  GROUP BY rewardsReceiptStatus''').fetchall()

# Communicate with Stakeholders
Construct an email or slack message that is understandable to a product or business leader who isn’t familiar with your day to day work. This part of the exercise should show off how you communicate and reason about data with others. Commit your answers to the git repository along with the rest of your exercise.

* What questions do you have about the data?
* How did you discover the data quality issues?
* What do you need to know to resolve the data quality issues?
* What other information would you need to help you optimize the data assets you're trying to create?
* What performance and scaling concerns do you anticipate in production and how do you plan to address them?

In [ ]:
brands.isna().sum()

In [ ]:
users.isna().sum()

In [ ]:
receipts_final.columns

Hi all,

I've been working on the analysis of finding the average spend and total number of items purchased for the receipts which rewardsReceiptStatus is **Accepted or Rejected**. I want to update my progress and clarify questions with you.

I found that the average spend where rewardsReceiptStatus is **Finished is 1244.37, Rejected is 19.54**, and the total number of items purchased for **Finished is 1364998, Rejected is 740. Both average spend and total number of items purchased where the rewardsReceiptStatus is FINISHED is a lot higher than REJECTED. 

Questions I want to clarify:

1. Data Definition Clarification For The Analysis
In the current analysis, I’m assuming that **Finished = Accepted** because I didn’t see any row of data where rewardsReceiptStatus is **Accepted**. Correct me if I am wrong, and I will update the analysis.

2. Data Format
The current data is not well structured. For example, there is lots of data under rewardsReceiptItemList, and I need to spend time restructuring the data frame for analysis. Is the format required in the product implementation? This type of data storage format would be cost a lot time for data wrangling in the future when we have millions of data. It would be helpful for us to spend time discussing restructuring the current data frame so that we could work more efficiently in the future.
I also want to discuss the data relationships




3. Missing Value and Outliers 
There are some missing values in Users, Brands, Receipts tables. There are especially a great amount of null values in rewardsReceiptItemList column. Could you let me know what the data collection process is, so that we can brainstorm methods to improve data quality, reduce the null value in the future. 

There are also lots of extreme values in the dataset. For example, a few transactions earned 6000+ points, and spent 4000+ dollars. Are these possible transactions, or is it incorrect information?

Some user_id is null when we join Receipts table and Users, this might due to the fact that users did not log into the account. It we be helpful for us to discuss some alternative methods to gather user information. Having sufficient customer/user data will help further customer analysis. Another data issue I want to address is that there are a lot of receipts in the data which brandcodes do not exist in the brands data. This could be problematic when we are doing analysis on brand level. 

Let me know what your thoughts are. We can schedule a meeting to clarify above questions and discuss further analysis.

Thanks!

Best Regards,
Shana
